#May Lia Khoironisa-Submission NLP
Dataset:Sentimental analysis from kaggle

Kriteria Submission
1.   Dataset bebas,min. 1000 sampel (V)
2.   LSTM dalam arsitektur model(V)
3.   Model sequential(V)
4.   Val set=20% (V)
5.   Embedding(V)
6.   Tokenizer(V)
7.   Akurasi min 75% (V)
8.   Implementasi callback

In [90]:
import numpy as np
from numpy import array
import pandas as pd
import tensorflow as tf
import random
import seaborn as sns


from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
data_tweet=pd.read_csv('/content/sentiment_tweets3.csv')
data_tweet=data_tweet[['message to examine','label (depression result)']]
data_tweet.head()


message to examine  label (depression result)
0  just had a real good moment. i missssssssss hi...                          0
1         is reading manga  http://plurk.com/p/mzp1e                          0
2  @comeagainjen http://twitpic.com/2y2lx - http:...                          0
3  @lapcat Need to send 'em to my accountant tomo...                          0
4      ADD ME ON MYSPACE!!!  myspace.com/LookThunder                          0

In [ ]:
x=data_tweet['message to examine']
y=data_tweet['label (depression result)']
print(x[:500],y[:500])

0      just had a real good moment. i missssssssss hi...
1             is reading manga  http://plurk.com/p/mzp1e
2      @comeagainjen http://twitpic.com/2y2lx - http:...
3      @lapcat Need to send 'em to my accountant tomo...
4          ADD ME ON MYSPACE!!!  myspace.com/LookThunder
                             ...                        
495    @kencasey maybe a new beginning on drawing wil...
496    right wll. i'm bored so i'm going to busy myse...
497    @brother_rat yep, if there's one thing they ar...
498                                 @caseymairex  awww! 
499                                sweet dreams twitter 
Name: message to examine, Length: 500, dtype: object 0      0
1      0
2      0
3      0
4      0
      ..
495    0
496    0
497    0
498    0
499    0
Name: label (depression result), Length: 500, dtype: int64


In [ ]:
#split data
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.20)

In [ ]:
#hitung jumlah train 80%
(unique,counts)=np.unique(y_train,return_counts=True)
frequencies=np.asarray((unique,counts)).T
print(frequencies)

[[   0 6389]
 [   1 1862]]


In [ ]:
#hitung jumlah validation 20%
(unique,counts)=np.unique(y_test,return_counts=True)
frequencies=np.asarray((unique,counts)).T
print(frequencies)

[[   0 1611]
 [   1  452]]


In [ ]:
#model
vocab_size=10000 #for tokenizer
embedding_dim=16 #model conf
max_length=100 #word emb
trunc_type='post'
padding_type='post'
oov_tok="<OOV>"

In [ ]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(x) #just for training

word_index = tokenizer.word_index
#train sequences and padding
training_sequences = tokenizer.texts_to_sequences(X_train)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
#test sequences ad padding
testing_sequences = tokenizer.texts_to_sequences(X_test)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

#convert to numpy array
training_padded=np.array(training_padded)
training_labels=np.array(y_train)

testing_padded=np.array(testing_padded)
testing_labels=np.array(y_test)

In [97]:
model = Sequential()
model.add(Embedding(vocab_size,embedding_dim, input_length=max_length))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(24, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
adam = Adam(learning_rate=0.01)

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(training_padded,training_labels,batch_size=128,epochs=100,validation_data=(testing_padded,testing_labels),callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)],verbose = 1)
#print model.summary()
print(model)

Epoch 1/100
65/65 [==============================] - 76s 614ms/step - loss: 0.0000e+00 - accuracy: 0.7702 - val_loss: 0.0000e+00 - val_accuracy: 0.7809


In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  


In [ ]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 100, 16)           160000    
_________________________________________________________________
bidirectional_5 (Bidirection (None, 300)               200400    
_________________________________________________________________
dense_10 (Dense)             (None, 24)                7224      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 25        
Total params: 367,649
Trainable params: 367,649
Non-trainable params: 0
_________________________________________________________________


In [ ]:
score, acc = model.evaluate(testing_padded, testing_labels, verbose = 2)
print("Test Score model ini adalah:", score)
print("Test Accuracy model ini adalah:", acc)

65/65 - 3s - loss: 0.0000e+00 - accuracy: 0.7809
Test Score model ini adalah: 0.0
Test Accuracy model ini adalah: 0.7809016108512878
